In [ ]:
# Evaluation demo for YOLO detections and color
# This notebook loads the trained YOLO model, runs inference on sample images, and shows predicted classes, boxes, and the dominant color per box.

import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from color_utils import compute_dominant_color_id, COLOR_ID_TO_NAME

DATASET_DIR = '/Users/tomas.komar/digitecus/fotos/dataset'
IMAGES_DIR = os.path.join(DATASET_DIR, 'images', 'val')
WEIGHTS = '/Users/tomas.komar/digitecus/fotos/runs/detect/stereo_objects/weights/best.pt'
CLASS_NAMES = ['any/unknown','can','duck','cup','sponge','ball','vegetable']

model = YOLO(WEIGHTS)

def show_image_with_preds(img_path):
	img = cv2.imread(img_path, cv2.IMREAD_COLOR)
	if img is None:
		print('Failed to read', img_path)
		return
	res = model.predict(img, imgsz=640, conf=0.25)[0]
	img_show = img.copy()

	for b in res.boxes:
		cls_id = int(b.cls.item())
		conf = float(b.conf.item())
		x1, y1, x2, y2 = map(int, b.xyxy[0].tolist())
		color_id = compute_dominant_color_id(img, (x1,y1,x2,y2))
		label = f"{CLASS_NAMES[cls_id]} {conf:.2f} | {COLOR_ID_TO_NAME.get(color_id, 'any/unknown')}"
		cv2.rectangle(img_show, (x1,y1), (x2,y2), (0,255,0), 2)
		cv2.putText(img_show, label, (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

	img_show_rgb = cv2.cvtColor(img_show, cv2.COLOR_BGR2RGB)
	plt.figure(figsize=(10,6))
	plt.imshow(img_show_rgb)
	plt.axis('off')
	plt.title(os.path.basename(img_path))
	plt.show()

# Demo: show a few images
val_images = [os.path.join(IMAGES_DIR, f) for f in os.listdir(IMAGES_DIR) if f.lower().endswith('.png')]
for p in val_images[:10]:
	show_image_with_preds(p)

